<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

## DataFrame Manipulation Lab with Chipotle Data

_Authors: Kiefer Katovich (SF)_

---

This lab is intended to cover a variety of skills for data manipulation in pandas with a challenging dataset.

In addition to python function-writing practice, you will be practicing multiple pandas EDA skills including:
- Data cleaning
- Grouping
- Data summarization and aggregation
- [Pandas split-apply-combine pattern](http://pandas.pydata.org/pandas-docs/stable/groupby.html)
- Basic plotting


In [76]:
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

sns.set_style('whitegrid')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

#### 1. Load the `chipotle.tsv` and examine the data.

In [77]:
chip_file = './datasets/chipotle.tsv'

The chipotle data is a .tsv, which means "tab separated value". This is just like a csv but the cells are separated by tabs. There is an argument in read_csv called delimiter, where you can specify the string that separates the tabs:

In [78]:
# A:
chip = pd.read_csv(chip_file, delimiter = '\t')

chip.head(10)

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98
5,3,1,Chicken Bowl,"[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sou...",$10.98
6,3,1,Side of Chips,NaN,$1.69
7,4,1,Steak Burrito,"[Tomatillo Red Chili Salsa, [Fajita Vegetables...",$11.75
8,4,1,Steak Soft Tacos,"[Tomatillo Green Chili Salsa, [Pinto Beans, Ch...",$9.25
9,5,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Pinto...",$9.25


In [79]:
np.arange(chip.shape[0])

array([   0,    1,    2, ..., 4619, 4620, 4621])

#### 2. Create a sub-id for each order-id

We have an identifier for each order already in `order_id`, but no unique identifier for each _sub-order_ within the overall order.

Use grouping and `.apply()` to assign sub-ids for orders.

In [80]:
# A:
def add_sub_order_id(df):
    df['sub_order_id'] = np.arange(df.shape[0]) 
    return df # making an array using the df passes shapes

chip = chip.groupby('order_id').apply(add_sub_order_id)
chip.head(10)

,order_id,quantity,item_name,choice_description,item_price,sub_order_id
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39,0
1,1,1,Izze,[Clementine],$3.39,1
2,1,1,Nantucket Nectar,[Apple],$3.39,2
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39,3
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98,0
5,3,1,Chicken Bowl,"[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sou...",$10.98,0
6,3,1,Side of Chips,NaN,$1.69,1
7,4,1,Steak Burrito,"[Tomatillo Red Chili Salsa, [Fajita Vegetables...",$11.75,0
8,4,1,Steak Soft Tacos,"[Tomatillo Green Chili Salsa, [Pinto Beans, Ch...",$9.25,1
9,5,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Pinto...",$9.25,0


#### 3. Clean up the price column 

We want the price column to be a numeric float value. Currently it is a string (and has the dollar sign in it).

In [81]:
# A:
def dollar_replace(price):
    x = price.replace('$','')
    return float(x)
chip['item_price'] = chip['item_price'].map(dollar_replace)
chip.head()

,order_id,quantity,item_name,choice_description,item_price,sub_order_id
0,1,1,Chips and Fresh Tomato Salsa,NaN,2.39,0
1,1,1,Izze,[Clementine],3.39,1
2,1,1,Nantucket Nectar,[Apple],3.39,2
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,2.39,3
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",16.98,0


#### 4. Make a new categorical column for broader item type

Currently we have many different item types. Make a new column that only has 5 different broad item types. You should have these types in the new column in your DataFrame:

    chips
    drink
    burrito
    taco
    salad
    
(Put the `bowl` items into `burrito` category).

In [82]:
# A:
def food_category(food):
    food = food.lower()
    
    if 'chips' in food:
        return 'chips'
    elif 'drink' in food:
        return 'drink'
    elif 'burrito' in food or 'bowl' in food:
        return 'burrito'
    elif 'taco' in food:
        return 'taco'
    else:
        return 'salad'

chip['item_category'] = chip['item_name'].map(food_category)

#### 5. Calculate the total price by `order_id` and add as a new as column `order_total_price`.

There are a variety of different ways you can tackle this problem. One way is a grouped apply on the price and then a merge by `order_id` with the total price.

Hints:

- Merging DataFrames with series doesn't work, you need to merge two DataFrames.
- A series object coming out of a groupby with an apply will have the groupby as potentially hierarchical indices. Using `reset_index()` will turn these back into columns and also convert to a DataFrame, which can be used to merge on.

In [83]:
# A:
order_prices =  chip.groupby(['order_id']).item_price.apply(np.sum).reset_index()
order_prices.rename(columns={'item_price':'order_total_price'}, inplace=True)

order_prices.head(10)


chip = chip.merge(order_prices, on='order_id').head(10)

#### 6. Make an `adjusted_item_price` column to account for multiple orders per row.

Some items have multiple orders per row, as indicated by the quantity. Adjust the price to account for the number of orders in a new column.

In [84]:
# A:
chip['adj_price'] = chip.item_price/chip.quantity
chip

,order_id,quantity,item_name,choice_description,item_price,sub_order_id,item_category,order_total_price,adj_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,2.39,0,chips,11.56,2.39
1,1,1,Izze,[Clementine],3.39,1,salad,11.56,3.39
2,1,1,Nantucket Nectar,[Apple],3.39,2,salad,11.56,3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,2.39,3,chips,11.56,2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",16.98,0,burrito,16.98,8.49
5,3,1,Chicken Bowl,"[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sou...",10.98,0,burrito,12.67,10.98
6,3,1,Side of Chips,NaN,1.69,1,chips,12.67,1.69
7,4,1,Steak Burrito,"[Tomatillo Red Chili Salsa, [Fajita Vegetables...",11.75,0,burrito,21.00,11.75
8,4,1,Steak Soft Tacos,"[Tomatillo Green Chili Salsa, [Pinto Beans, Ch...",9.25,1,taco,21.00,9.25
9,5,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Pinto...",9.25,0,burrito,13.70,9.25


#### 7. What is the min, max, count, mean and standard deviation of price for each unique item in  `item_name`?

Pivot table works well for this. You can pass multiple aggregation functions into the `aggfunc` argument.

The count won't just be the length of each order's subset - there are multiple orders per row sometimes (evidenced in the quantity column).

In [99]:
# A:

item_summary_stats = pd.pivot_table(chip, values='adj_price', columns=['item_name'],
               aggfunc=[np.max, np.min, np.mean, np.std])

item_summary_stats.rename(columns = {'amax':'max', 'amin':'min'}, inplace = True)
item_summary_stats = item_summary_stats.reset_index()

item_summary_stats


item_summary_counts = pd.pivot_table(chip, values = 'quantity', columns = ['item_name'],
                                     aggfunc=[np.sum])
item_summary_counts = item_summary_counts.reset_index()
item_summary_counts


# Get the sum of the quantity column by item name:
# Merge the two statistic dataframes together

index          sum                               \
item_name           Chicken Bowl Chips and Fresh Tomato Salsa   
0          quantity            3                            1   

                                                                       \
item_name Chips and Tomatillo-Green Chili Salsa Izze Nantucket Nectar   
0                                             1    1                1   

                                                        
item_name Side of Chips Steak Burrito Steak Soft Tacos  
0                     1             2                1

In [100]:
item_summary_counts.pivot(columns='item_name', values='quantity')


KeyError: 'quantity'

#### 8. Plot the mean price of items against the count (popularity).

You have this info in your summary table from the previous question.

In [10]:
# A:

#### 9. Plot the max price of items against the count (popularity).

In [11]:
# A:

### 10. Calculate the mean of adjusted price per broad category.

You can handle these with a single function if you want, or another way if you prefer.

Just FYI, apply functions can have keyword arguments that you pass in when you call the apply chained to the groupby.

for example:

```python
def my_applier(df, my_kwarg='placeholder'):
    df['newcol_'+placeholder] = 1.
    return df
    
data = data.groupby('variable').apply(my_applier, my_kwarg='colsuffix').reset_index(drop=True)
```


In [12]:
# A:

#### 11. Make a barplot of your price mean by the broad type category.


In [13]:
# A:

#### 12. [Challenge] Parse the `choice_description` column into two new columns: `order_customizations` and `order_customization_id`

Here is what your inputs and outputs would look like for a hypotheical section of the DataFrame (I'm just showing some of the columns to give you an idea of what the output format will be):

**Input:**

```python
                                  choice_description     item_name  order_id  \
0                                       [Clementine]          Izze         1   
1  [Red Salsa, [Black Beans, Guacamole, Sour Cream]]  Chicken Bowl         1   

   sub_order_id  
0             1  
1             2
```

**Output:**

```python
   order_customization_id order_customizations  \
0                       0           Clementine   
1                       0            Red Salsa   
2                       1          Black Beans   
3                       2            Guacamole   
4                       3           Sour Cream   

                                  choice_description     item_name  order_id  \
0                                       [Clementine]          Izze         1   
1  [Red Salsa, [Black Beans, Guacamole, Sour Cream]]  Chicken Bowl         1   
2  [Red Salsa, [Black Beans, Guacamole, Sour Cream]]  Chicken Bowl         1   
3  [Red Salsa, [Black Beans, Guacamole, Sour Cream]]  Chicken Bowl         1   
4  [Red Salsa, [Black Beans, Guacamole, Sour Cream]]  Chicken Bowl         1   

   sub_order_id  
0             1  
1             2  
2             2  
3             2  
4             2 
```

Hints:

- Remember you can write your own function and pass it into apply. In this case there will be one item per group, since we have to do this parsing for every row, but you may be able to get a solution with `.iterrows()` if you want to try that out.
- Within a function that you are passing into `.apply()`, you can create a _new DataFrame and return that._ This is one of the things that makes apply so powerful, since you can essentially perform any operations you want on a subset of your original DataFrame as long as you return DataFrames/groups that can be recombined.

- *Your output dataframe will be very long as there will be a row for every item in the item_name column.  Expect there to be alot of repeating information betweens rows, but they should not be exactly identical.*

**Note: the function may take awhile to complete. `apply` isn't that efficient with complicated custom operations per row like this.**

In [14]:
# A: